<a href="https://colab.research.google.com/github/JandreiSS/Analise-Fundamentalista/blob/master/DataFrame_bolsa.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import string
import requests
import warnings
warnings.filterwarnings('ignore')
from concurrent.futures import ThreadPoolExecutor, as_completed
from tqdm import tqdm

In [2]:
header = {
  "User-Agent": "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/50.0.2661.75 Safari/537.36",
  "X-Requested-With": "XMLHttpRequest"
}

In [3]:
url = 'http://fundamentus.com.br/resultado.php'

In [4]:
r = requests.get(url, headers=header)

In [5]:
df = pd.read_html(r.text)[0]

In [6]:
df = df.loc[:,['Papel']]

In [7]:
def get(url, timeout):
    return requests.get(url, headers=header, timeout = timeout)

In [8]:
def requestUrls(urls, timeout = 5):
    with ThreadPoolExecutor(max_workers = 3) as executor:
        agenda = { executor.submit(get, url, timeout): url for url in urls }

        for tarefa in as_completed(agenda):     
            try:
                conteudo = tarefa.result()
            except Exception as e:
                print ("\nNão foi possível fazer a requisição! \n{}".format(e))
            else:
                yield conteudo

In [9]:
urls = 'https://www.fundamentus.com.br/detalhes.php?papel=' + df['Papel'].values

In [10]:
requisicoes = requestUrls(urls, timeout=60)

In [11]:
lista_empresas = []

In [12]:
total_ticket = len(df)

In [13]:
for requisicao in tqdm(requisicoes, unit=' requisições', desc='Requisitando dados: ', disable=False, total=total_ticket, ncols=100):
    # codigo = requisicao.status_code
    # url = requisicao.url
    conteudo = requisicao.content
    lista_empresas += [conteudo]
    # print (f'{codigo}: {url}')

Requisitando dados: 100%|███████████████████████████████| 887/887 [06:10<00:00,  2.40 requisições/s]


# Transformar os dados para as colunas, para depois exportar em CSV

In [14]:
colunas = [
    'Papel',
    'Empresa',
    'Setor',
    'Subsetor',
    'Cotação',
    'Valor da Firma',
    'Valor de Mercado',
    'Número de Ações',
    'P/L',
    'Div. Yield',
    'EV/EBITDA',
    'EV/EBIT',
    'ROE'
]

In [15]:
tabela_base = pd.DataFrame(columns=colunas)

In [16]:
total_lista = len(lista_empresas)

In [17]:
for key in tqdm(range(total_lista), unit=' dados', desc='Tratando dados: ', disable=False, total=total_lista, ncols=100):
    dados_base = pd.read_html(lista_empresas[key], thousands='.', decimal=',')
    papel = dados_base[0][1][0]
    empresa = dados_base[0][1][2]
    setor = dados_base[0][1][3]
    subsetor = dados_base[0][1][4]
    cotacao = dados_base[0][3][0]
    valor_da_firma = dados_base[1][1][1]
    valor_de_mercado = dados_base[1][1][0]
    n_acoes = dados_base[1][3][1]
    p_l = dados_base[2][3][1]
    div_yield = dados_base[2][3][8]
    ev_ebitda = dados_base[2][3][9]
    ev_ebit = dados_base[2][3][10]
    roe = dados_base[2][5][8]
    to_append = [papel, empresa, setor, subsetor, cotacao, valor_da_firma, valor_de_mercado, n_acoes, p_l, div_yield, ev_ebitda, ev_ebit, roe]
    tabela_base_length = len(tabela_base)
    tabela_base.loc[tabela_base_length] = to_append


Tratando dados: 100%|█████████████████████████████████████████| 887/887 [00:42<00:00, 20.80 dados/s]


In [18]:
tabela_base

,Papel,Empresa,Setor,Subsetor,Cotação,Valor da Firma,Valor de Mercado,Número de Ações,P/L,Div. Yield,EV/EBITDA,EV/EBIT,ROE
0,VNET3,CIELO ON NM,Serviços Financeiros Diversos,Serviços Financeiros Diversos,0.00,-,0,2716820000,0.00,"0,0%",-,-,"13,0%"
1,CLAN3,CLARION ON,Alimentos,Grãos e Derivados,0.00,-,0,158703000,0.00,"0,0%",-,-,"-1,0%"
2,CFLU4,COCA COLA PN,NaN,NaN,1000.00,-,0,0,0.00,"0,0%",-,-,"32,2%"
3,CSTB3,COMPANHIA SIDERÚRGICA DE TUBARÃO ON,NaN,NaN,150.00,-,0,0,0.00,"0,0%",-,-,"20,1%"
4,CLSC6,CELESC PNB N2,Energia Elétrica,Energia Elétrica,0.00,-,0,38572000,0.00,"0,0%",-,-,"22,9%"
...,...,...,...,...,...,...,...,...,...,...,...,...,...
882,CEPE3,CELPE ON,Energia Elétrica,Energia Elétrica,24.53,6818230000,1830230000,74612000,1895.99,"0,0%",269.96,326.16,"0,1%"
883,CEPE6,CELPE PNB,Energia Elétrica,Energia Elétrica,20.00,6480240000,1492240000,74612000,1545.85,"0,0%",256.58,309.99,"0,1%"
884,CEPE5,CELPE PNA,Energia Elétrica,Energia Elétrica,27.00,7002520000,2014520000,74612000,2086.90,"0,0%",277.26,334.98,"0,1%"
885,BRAP4,BRADESPAR S/A PN N1,Holdings Diversificadas,Holdings Diversificadas,39.98,13940200000,13914400000,348034000,2467.09,"2,0%",-798.13,-798.13,"0,1%"


In [19]:
# tabela_base.to_csv('tabela_dados.csv', index=False)